In [ ]:
!pip install llama-index

!pip install llama-index-vector-stores-mongodb

!pip install llama-index-embeddings-openai

!pip install pymongo

!pip install datasets

!pip install pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.0/670.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%env OPENAI_API_KEY=your_api_key

env: OPENAI_API_KEY=your_api_key


In [ ]:
import pandas as pd
import uuid  # Optional for random UUIDs
import re

from datasets import load_dataset

dataset = load_dataset("aakash0017/It-support-synthetic-data")

# Define regular expression pattern (slightly modified)
pattern = r"(?<=### Human: )(.*?)(?=### Assistant: )"  # Use non-greedy match

# Create empty lists to store questions and answers
questions = []
answers = []

# Extract data using regular expression
for row in dataset["train"]:
  match = re.search(pattern, row["text"])
  if match:
    questions.append(match.group(1).strip())  # Strip trailing whitespaces
    answers.append(row["text"].split("### Assistant: ")[-1])

# Create DataFrame with extracted data
dataset_df = pd.DataFrame({"question": questions, "answer": answers})

# Generate unique IDs with leading zeros for consistent formatting
dataset_df["ticket-number"] = "IDC" + dataset_df.apply(lambda row: str(uuid.uuid4())[:8], axis=1).str.zfill(8)

# Add text_content column
dataset_df["text_content"] = dataset_df["question"] + " " + dataset_df["answer"]

# Reorder columns to make "ticket-number" the first column
reordered_cols = ["ticket-number", "question", "answer", "text_content"]
dataset_df = dataset_df[reordered_cols]
dataset_df.head(10)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/15908 [00:00<?, ? examples/s]

,ticket-number,question,answer,text_content
0,IDC6f7b8ba2,What is the newest learning management system ...,Blackboard Learn.,What is the newest learning management system ...
1,IDC13e56c0e,Does Boston University offer a laptop loan pro...,"Yes, they have a laptop loaner program for act...",Does Boston University offer a laptop loan pro...
2,IDC1be7257c,What is Blackboard Learn?,Blackboard Learn is the primary learning manag...,What is Blackboard Learn? Blackboard Learn is ...
3,IDC6bd055bb,What are the key features of Blackboard Learn?,The key features of Blackboard Learn include i...,What are the key features of Blackboard Learn?...
4,IDC399daaaf,What to expect from Blackboard Learn?,"Blackboard Learn is normally available 24/7, e...",What to expect from Blackboard Learn? Blackboa...
...,...,...,...,...
7609,IDC4fc1d09d,"What happens when you insert a ""More"" tag?","When you insert a ""More"" tag, all the content ...","What happens when you insert a ""More"" tag? Whe..."
7610,IDC21351a6d,"What is the purpose of the ""Headline Thumbnail...","The purpose of the ""Headline Thumbnails"" plugi...","What is the purpose of the ""Headline Thumbnail..."
7611,IDC8e6c8d43,What are headline thumbnails used for?,Headline thumbnails are used to add an interes...,What are headline thumbnails used for? Headlin...
7612,IDCd47033fd,How do I add a headline thumbnail to my news p...,"To add a headline thumbnail to your news post,...",How do I add a headline thumbnail to my news p...


In [ ]:
""" For testing purpose. Takes 10 rows from the data so it takes less time.
import pandas as pd
import re

from datasets import load_dataset

# Assuming you have already run the code that creates the DataFrame (df) with questions and answers

# Extract only the first 10 rows
dataset_df = dataset_df.head(10)

dataset_df.head(100)
"""

,ticket-number,question,answer,text_content
0,IDC79659407,What is the newest learning management system ...,Blackboard Learn.,What is the newest learning management system ...
1,IDCfc1b32d0,Does Boston University offer a laptop loan pro...,"Yes, they have a laptop loaner program for act...",Does Boston University offer a laptop loan pro...
2,IDC5385384b,What is Blackboard Learn?,Blackboard Learn is the primary learning manag...,What is Blackboard Learn? Blackboard Learn is ...
3,IDCb586e780,What are the key features of Blackboard Learn?,The key features of Blackboard Learn include i...,What are the key features of Blackboard Learn?...
4,IDC1309c0a1,What to expect from Blackboard Learn?,"Blackboard Learn is normally available 24/7, e...",What to expect from Blackboard Learn? Blackboa...
5,IDCf1f27cca,How do I get started with Blackboard Learn?,"To get started with Blackboard Learn, faculty ...",How do I get started with Blackboard Learn? To...
6,IDC53714075,What does the IT Help Center provide?,The IT Help Center provides centralized techno...,What does the IT Help Center provide? The IT H...
7,IDC87909b25,How do I contact the IT Help Center?,You can contact the IT Help Center online or b...,How do I contact the IT Help Center? You can c...
8,IDCbd8115e6,What services does the IT Help Center offer?,The IT Help Center offers a variety of service...,What services does the IT Help Center offer? T...
9,IDCa2be0617,Do I need a current BU login name to use some ...,"Yes, some services require a current BU login ...",Do I need a current BU login name to use some ...


In [ ]:
from llama_index.core.settings import Settings

from llama_index.llms.openai import OpenAI

from llama_index.embeddings.openai import OpenAIEmbedding

embed_model=OpenAIEmbedding(model="text-embedding-3-small",dimensions=256)

llm = OpenAI(model="gpt-4-turbo")

Settings.llm=llm

Settings.embed_model=embed_model


In [ ]:
import json
from llama_index.core import Document
from llama_index.core.schema import MetadataMode

# Convert the DataFrame to a JSON string representation
documents_json = dataset_df.to_json(orient='records')
# Load the JSON string into a Python list of dictionaries
documents_list = json.loads(documents_json)

llama_documents = []

for document in documents_list:

  # Value for metadata must be one of (str, int, float, None)
  document["ticket-number"] = json.dumps(document["ticket-number"])
  document["question"] = json.dumps(document["question"])
  document["answer"] = json.dumps(document["answer"])



  # Create a Document object with the text and excluded metadata for llm and embedding models
  llama_document = Document(
      text=document["text_content"],
      metadata=document,
      excluded_llm_metadata_keys=["question", "answer"],
      excluded_embed_metadata_keys=["question", "answer"],

      metadata_template="{key}=>{value}",
      text_template="Metadata: {metadata_str}\n-----\nContent: {content}",
      )

  llama_documents.append(llama_document)

# Observing an example of what the LLM and Embedding model receive as input
print(
    "\nThe LLM sees this: \n",
    llama_documents[0].get_content(metadata_mode=MetadataMode.LLM),
)
print(
    "\nThe Embedding model sees this: \n",
    llama_documents[0].get_content(metadata_mode=MetadataMode.EMBED),
)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter()
nodes = parser.get_nodes_from_documents(llama_documents)

for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding


In [ ]:
#set the uri before running this cell.
import pymongo
from google.colab import userdata


def get_mongo_client(mongo_uri):
  """Establish connection to the MongoDB."""
  try:
    client = pymongo.MongoClient(mongo_uri)
    print("Connection to MongoDB successful")
    return client
  except pymongo.errors.ConnectionFailure as e:
    print(f"Connection failed: {e}")
    return None

mongo_uri = userdata.get('MONGO_URI')
if not mongo_uri:
  print("MONGO_URI not set in environment variables")

mongo_client = get_mongo_client(mongo_uri)

DB_NAME="itdata"
COLLECTION_NAME="it_support_data"

db = mongo_client[DB_NAME]
collection = db[COLLECTION_NAME]

Connection to MongoDB successful


In [ ]:
# To ensure we are working with a fresh collection
# delete any existing records in the collection

collection.delete_many({})


DeleteResult({'n': 0, 'electionId': ObjectId('7fffffff000000000000030b'), 'opTime': {'ts': Timestamp(1714584301, 16), 't': 779}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1714584301, 16), 'signature': {'hash': b'[O\xaa\xe8\x1dCM.\xba\xe4\x18\x9b^\xef=\xa7\xb0K|\x0b', 'keyId': 7311276111621521410}}, 'operationTime': Timestamp(1714584301, 16)}, acknowledged=True)

In [ ]:
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch

vector_store = MongoDBAtlasVectorSearch(mongo_client, db_name=DB_NAME, collection_name=COLLECTION_NAME, index_name="vector_index")
vector_store.add(nodes)


['47041c9f-a200-4f93-89cc-ada739243b36',
 '629e7246-19fa-44c3-848b-20e15246cde2',
 '1dc2e25c-fcb9-44d2-aa4b-60a882afd341',
 'a2894e0f-b4bd-48e4-ba69-147ce6e657ae',
 'cee88a8a-fd63-4ac4-8ea8-699040cfe8cf',
 '6d34fae8-5814-420c-b262-43e8de6bc7a6',
 '8c1e6efa-4121-42a8-b6a5-e12349ab6b3e',
 '3948acf2-c5b2-4584-868c-b717a301bba2',
 '6bd215e1-e92e-441b-829d-b252e5692a79',
 '85ea4390-69b7-4308-a836-68b1499d16c0',
 'c835c130-df08-4cfa-a163-80d3679f222d',
 '53703825-2587-456b-98ce-f2b2458528dd',
 '4f6e1d47-deca-4746-add0-40e2e9db7d1f',
 '4105f68d-af76-4426-8c2c-463273a0ee83',
 'bc428430-f3f9-4b21-8679-3895fc2a60b1',
 '385ba2ff-77b7-4e95-9b79-29322fa25c2e',
 '5f764e06-3965-4637-9e26-3afecf6ed64c',
 'e37ac057-8633-464f-956f-86db7a7a2d52',
 '6b0e08f5-01e6-4b84-b87d-e91db9a799ec',
 '78b2baba-ddce-4601-95f5-9dfb69a1bb40',
 'f5cd1119-a552-4463-b371-531fe1a5b0aa',
 '78bc8901-075b-4c55-b839-acdd87fad82a',
 'e6999f37-2544-4e03-9069-a233cdfb6e09',
 '4e880444-697e-4307-babf-6d37ff5a69dd',
 '1454c553-971e-

In [ ]:
from llama_index.core import VectorStoreIndex, StorageContext
index = VectorStoreIndex.from_vector_store(vector_store)

In [ ]:

import pprint
from llama_index.core.response.notebook_utils import display_response

query_engine = index.as_query_engine(similarity_top_k=3)
query = "Do I need a current BU login name to use some services??"
response = query_engine.query(query)
display_response(response)
pprint.pprint(response.source_nodes)


**`Final Response:`** Empty Response

[]
